# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 18.02.2022 16:10:59


In [2]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
using BoSSS.Application.SipPoisson;

In [4]:
wmg.Init("LinslvPerf_ConstPoissonMpi1");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'LinslvPerf_ConstPoissonMpi1'.
Opening existing database 'C:\BoSSS-Databases\LinslvPerf_ConstPoissonMpi1'.


In [5]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,ExecutionTime,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]",,,,,,,,,,
1,BoSSS.Application.BoSSSpad.SlurmClient,X:\bosss_deploy,False,Lb2-specialPrj,dotnet,,[ X: == /work/scratch/fk69umer ],fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/bosss_deploy,special00006,<null>,False
2,BoSSS.Application.BoSSSpad.SlurmClient,X:\jenkins\deploy,False,Lb2-specialPrj-Jenkins,dotnet,,[ X:\jenkins\databases == /work/scratch/fk69umer/jenkins/databases ],fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/jenkins/deploy,special00006,<null>,False


Queue which will be used:

In [6]:
GetDefaultQueue()

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]"


## Utility definitions

In [7]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [8]:
wmg.Grids

DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\LinslvPerf_ConstPoissonMpi1'..
DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\jenkins\databases\LinslvPerf_ConstPoissonMpi1'..


Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

In [9]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    //grids[cnt] = null;
    //foreach(IGridInfo grd in wmg.Grids) { 
    //    bool check = grd.Name.Contains(string.Format("_J"+ J));
    //    if(check) {
    //      grids[cnt] = grd;
    //    }
    //}
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null) { // 
        Console.WriteLine($"Creating grid with {J} = {Res*5}x{Res}x{Res} cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
    
        g.DefineEdgeTags(delegate (double[] X) {
            if(Math.Abs(X[0] - 0.0) <= 1.0e-6)
                return BoundaryType.Dirichlet.ToString();
            else
                return BoundaryType.Neumann.ToString();
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Creating grid with 320 = 20x4x4 cells.
Grid Edge Tags changed.
Creating grid with 2560 = 40x8x8 cells.
Grid Edge Tags changed.
Creating grid with 20480 = 80x16x16 cells.
Grid Edge Tags changed.
Creating grid with 69120 = 120x24x24 cells.
Grid Edge Tags changed.
Creating grid with 163840 = 160x32x32 cells.
Grid Edge Tags changed.
Creating grid with 552960 = 240x48x48 cells.
Grid Edge Tags changed.
Creating grid with 1310720 = 320x64x64 cells.
Grid Edge Tags changed.


In [10]:
grids

#0: { Guid = ee73f25c-02ed-4806-a982-5ab2f3408920; Name = LinslvPerf_ConstPoissonMpi1_J320; Cell Count = 320; Dim = 3 }
#1: { Guid = fc9f8a1c-229a-478d-b273-cce353cadf38; Name = LinslvPerf_ConstPoissonMpi1_J2560; Cell Count = 2560; Dim = 3 }
#2: { Guid = f10fe4c2-808d-4f2c-8bc2-725f49c21dd7; Name = LinslvPerf_ConstPoissonMpi1_J20480; Cell Count = 20480; Dim = 3 }
#3: { Guid = 03cb3888-64d3-4d1f-9536-e83931be87ff; Name = LinslvPerf_ConstPoissonMpi1_J69120; Cell Count = 69120; Dim = 3 }
#4: { Guid = fe1cbd73-007f-4796-b0f6-518364687cce; Name = LinslvPerf_ConstPoissonMpi1_J163840; Cell Count = 163840; Dim = 3 }
#5: { Guid = 17919a5d-c1c8-4889-b5c4-04c108c07662; Name = LinslvPerf_ConstPoissonMpi1_J552960; Cell Count = 552960; Dim = 3 }
#6: { Guid = a238ac8f-5439-4657-a401-4ec7ed2a1185; Name = LinslvPerf_ConstPoissonMpi1_J1310720; Cell Count = 1310720; Dim = 3 }


## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [11]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [12]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [13]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.19866933079506122

In [14]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

-0.8390715290764524

### Setup of Parameter Study

Polynomial degrees to test:

In [15]:
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5}; // reduced options for dev.

Solvers which we want to instrument:

In [16]:
using BoSSS.Solution.AdvancedSolvers;

In [17]:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    //LinearSolverCode.classic_mumps, 
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_ILU
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [18]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [19]:
List<SipControl> controls = new List<SipControl>();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name)) {
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add(ctrl);
       
    
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver           = solver_name.GetConfig();
    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    
    
    ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.METIS;
    //ctrl.GridPartType = BoSSS.Foundation.Grid.GridPartType.directHilbert;
}
}
}


(22,23): error CS1061: 'ISolverFactory' does not contain a definition for 'SolverCode' and no accessible extension method 'SolverCode' accepting a first argument of type 'ISolverFactory' could be found (are you missing a using directive or an assembly reference?)

(23,23): error CS1061: 'ISolverFactory' does not contain a definition for 'NoOfMultigridLevels' and no accessible extension method 'NoOfMultigridLevels' accepting a first argument of type 'ISolverFactory' could be found (are you missing a using directive or an assembly reference?)

(24,23): error CS1061: 'ISolverFactory' does not contain a definition for 'TargetBlockSize' and no accessible extension method 'TargetBlockSize' accepting a first argument of type 'ISolverFactory' could be found (are you missing a using directive or an assembly reference?)

(25,23): error CS1061: 'ISolverFactory' does not contain a definition for 'ConvergenceCriterion' and no accessible extension method 'ConvergenceCriterion' accepting a first arg

Cell not executed: compilation error

In [ ]:
//foreach(var s in wmg.Sessions.Where(s => s.CreationTime.Day == 13))
//   s.Delete(true);

In [ ]:
/*
// Save results on local workstation to reporduce particular cases:
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\ipPoisson\bin\Release\net5.0\PoissonBench";
foreach(var c in controls) {
    string name = c.SessionName;
    Console.Write(name + ": ");
    try {
        
        var txt = c.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [ ]:
controls.Count()

## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(); // run in the default queue for this project
}

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail.LatestDeployment.DeploymentDirectory.FullName);

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => (Si.SuccessfulTermination == false)
                                        || (Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

## Create Table for Post-Processing

(moved to post-processing worksheet)

In [ ]:
/*
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}
*/

In [ ]:
/*
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
*/

In [ ]:
/*
var FullSessTab = wmg.SessionTable;

// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = FullSessTab.GetColumnNames().ToList();
AllCols.Remove("Session");
FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());
*/

In [ ]:
//foreach(var s in AllCols) Console.WriteLine(s);

In [ ]:
//foreach(var s in wmg.Sessions[0].KeysAndQueries.Keys) Console.WriteLine(s + " \t = " + wmg.Sessions[0].KeysAndQueries[s]);

In [ ]:
/*var SubTab = FullSessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");
*/

In [ ]:
//SubTab.Print();

In [ ]:
/*
// Filename
var now               = DateTime.Now;
FullSessTab.TableName = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath        = FullSessTab.TableName + ".json";
SubTab.SaveToFile(wmg.CurrentProject + ".json");
SubTab.ToCSVFile(wmg.CurrentProject + ".csv");
*/